This is just exact diagonalization from scratch instead of using qutip. **The difference with qpc_qutip_fermionic is that here we are not including the fermionic, redundant degrees of freedom as in that case.  We are just building the basis for spinless fermion in the computational basis.Meaning our hilbert space is of dimension $2L_{qpc}$**

These functions are the same I have on the scattering region analysis notebook

In [30]:
import sys
import gc

import numpy as np
import pandas as pd

sys.path.append('../../scripts') 
import qutip
from qutip_tools import *

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import h5py
import json
import matplotlib 
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os
from ast import literal_eval

In [31]:
# define basic figure sizes
bwidth=7
bheight=5
base_font = 20

matplotlib.rcParams.update({'font.size': base_font,"legend.fontsize":base_font-4 ,"font.family": "serif", 
                            "font.serif": "CMU Serif, Times New Roman",
                           'text.usetex' : True })

sns.set_palette("colorblind")


In [32]:
L_qpc = 100
L = L_qpc + 2
Omega = 0.3
t = 0.01
J_prime = 1
bond_index = int(L_qpc/2)  # int(L_qpc/2)
J = np.ones(L_qpc) # QPC hopping
J[bond_index] = J_prime  
K0 = 0.9*np.pi/2
centered_at = 30 # initial position of wavepacket
Delta = 7.0 # spread of wavepacket
maxt_time =  L_qpc/(2 * J[0] * np.sin(K0)) + 1
N_timepoints = 800
x = np.arange(0,L_qpc)  # latice sites

ddot = "momentum" # can be free or momentum which is set by k0 based on af, bf
phi = 0 # initial phase of the qubit
# if its "free" af, bf will be the initial conditions
af = 0.3*np.pi # theta bloch angle at the time of hitting
print(np.cos(af/2)**2)

fig_route = "../../plots/perturbative/"
data_route = "../../data/exact_diag_new/L={}/".format(L_qpc)


Vg = 2 * J[1]* np.sin(K0)
L_qpc/ Vg

0.7938926261462367


50.623256289400146

# Functions

In [33]:
def create_hamiltonians(L, T, Bond):
    # creates the decoupled and interacting hamiltonians 
    
    # L_qpc = qpc lattice sites
    # T = qubit hopping
    # BOnd = index for bond locatin
    H_matrix = np.zeros((2*L,2*L))
    # fill in the dd hopping 
    d_indices= kth_diag_indices(H_matrix,1)
    H_matrix[d_indices] = -T
    
    # fill in the QPC hopping
    d_indices= kth_diag_indices(H_matrix,2)
    H_matrix[d_indices] = -J[0]
    
    # when qpc and qubit hop a the same time there is no contribution
    d_indices= kth_diag_indices(H_matrix,1)
    odd_inds = (d_indices[0][1::2], d_indices[1][1::2])
    H_matrix[odd_inds] = 0
    
    # save the free hamiltonian for later use
    Hdeco = H_matrix.copy()
    
    # Fill in the interaction at the bond
    H_matrix[2*Bond,2*(Bond+1)] = H_matrix[2*Bond,2*(Bond+1)]+ Omega
    
    # Now the elemets below the diagonal
    for i in range(0,2*L):
        for j in range(i + 1, 2*L):
            H_matrix[j, i] = H_matrix[i, j]
            Hdeco[j, i] = Hdeco[i, j]
            
    return H_matrix, Hdeco

    
def get_DD_init_for_fixed_k(k_prime):
    # calculated the initial conditions of the DD such that, when the QPC hits the bond
    # its state is the same as that of a DD initialized localized in the first site when 
    # the QPC for that case hits the bond with an average momentum k0=pi/2
    # k_prime: float. The momentum of the qpc particle
     
    alpha0 = np.cos( (t*bond_index)/(2*J[0])*(1/np.sin(k_prime) - 1) )
    beta0 = - 1j*np.sin( (t*bond_index)/(2*J[0])*(1/np.sin(k_prime) - 1) )
                        
    return alpha0, beta0

def get_bands(Eigen_energies, Eigen_vectors, Minus_indices, Plus_indies):
    # separates eigenvectors and eigenvalues into plus/minus enegy bands
    # and then sorts by enegy magnitude
    
    # minus band
    Energies_m = Eigen_energies[Minus_indices]
    States_m = Eigen_vectors[:,Minus_indices]
    # sort by magnitude
    Energies_m, States_m = mag_sort(Energies_m, States_m)
    
    # plus band
    Energies_p = Eigen_energies[Plus_indies]
    States_p = Eigen_vectors[:,Plus_indies]
    # sort by magnitude
    Energies_p, States_p = mag_sort(Energies_p, States_p)
    
    return Energies_m, States_m, Energies_p, States_p

def get_DD_init_for_momentum(k_prime,θf):
	# calculated the initial conditions of the DD such that, when the QPC hits the bond
	# its state is the same given by thetaf and follows an orbit between 0 a 1 with fixed phi
    # Here we achieve this by shfiting time appropriately
	# k_prime: float. The momentum of the qpc particle

    τ = (bond_index-centered_at)/(2*J[0]*np.sin(k_prime))
    alpha0 = np.cos(τ*t)*np.cos(0.5*θf) + 1j* np.sin(τ*t)*np.sin(0.5*θf)
    beta0 = np.cos(τ*t)*np.sin(0.5*θf) + 1j*np.sin(τ*t)*np.cos(0.5*θf)

    return alpha0, beta0

def get_DD_init_for_fixed_orbit(k_prime,θf):
	# calculated the initial conditions of the DD such that, when the QPC hits the bond
	# its state is the same given by thetaf and follows an orbit between 0 a 1 with fixed phi
    # Here we achieve this by shfiting time appropriately
	# k_prime: float. The momentum of the qpc particle
	dist = bond_index - centered_at 
	τ0t = - 0.5*θf + t*dist/(2*J[0]*np.sin(k_prime))
	alpha0 = np.cos(τ0t)
	beta0 = -1j*np.sin(τ0t)                  
	return alpha0, beta0

def get_DD_init_for_fixed_k(k_prime):
    # calculated the initial conditions of the DD such that, when the QPC hits the bond
    # its state is the same as that of a DD initialized localized in the first site when 
    # the QPC for that case hits the bond with an average momentum k0=pi/2
    # k_prime: float. The momentum of the qpc particle
    dist = bond_index - centered_at
    alpha0 = np.cos( (t*dist)/(2*J[0])*(1/np.sin(k_prime) - 1) )
    beta0 = - 1j*np.sin( (t*dist)/(2*J[0])*(1/np.sin(k_prime) - 1) )
                        
    return beta0, alpha0



In [34]:

def schmidt_qubit_qpc(c_eigs):
    # calculates the schmidt decomposition between the QPC and qubit for some stae
    # given how we build our hamiltonian, the 0qubit-eigenstates are in the even indices
    # while the 1qubit ones are in the odd indices
    # c_eigs is the state vector

    # arange in MPS form for SVD
    col_1 = np.asarray(c_eigs[0::2]).reshape(-1,1)
    col_2 =  np.asarray(c_eigs[1::2]).reshape(-1,1)
    psi_mat = np.concatenate((col_1, col_2), axis=1)
    U, S, Vh = np.linalg.svd(psi_mat)

    return U, S, Vh

def get_QPC_occupations(Psi):
    # apply the density opperator to the full composite Psi and return the
    # densities a teach of the QPC sites
    occ_list = []

    for j in range(0,L_qpc):
        basis_j = np.zeros(L_qpc)
        basis_j[j] = 1
        # create density operator for the jth qoc lattice
        ketbra = np.outer(basis_j, np.conj(basis_j))
        # tensor with the qubit identity
        Nj_op = np.kron(ketbra, np.eye(2))
        occ_list.append(np.vdot(Psi, Nj_op @ Psi).real)
    return np.asarray(occ_list)

def get_qubit_occupations(Psi):
    # apply the density opperator to the full composite Psi and return the
    # densities a teach of the QPC sites
    occ_list = []

    # create density operator for the 0th qubit state 
    ketbra = np.outer([1,0], np.conj([1,0]))
    # tensor with the qubit identity
    Nj_op = np.kron(np.eye(L_qpc), ketbra)
    return np.vdot(Psi, Nj_op @ Psi).real



In [57]:

def time_evolve(Psi0, Tau, states_, energies_):
    # time evolve a state up psi0 to time Tau with eigenstates states_m for anti-band
    # and states_p for symmetric band
    psi_t = np.zeros(len(Psi0)) + 0j
    for k in range(0,len(energies_)):
        psi_t += np.exp(-1j*Tau*energies_[k])*np.dot( np.conj(states_[:,k]), Psi0)*states_[:,k]

    return psi_t

"""
def get_reduced_density_matrix(Psi, NN):
    # calcualtes the partial trace from a STATEVECTOR Psi
    # NOT from the density matrix
    # Psi: array with the wavefunction
    # NN: integer with the total size of the lattice

    n = (NN-2) # QPC sites
    m = 2 # Double dot sites
    # get density matrix as a sparse array
    ρ = np.outer(Psi, np.conj(Psi.T))
    # trace out QPC sites and return reduced rho for DD as Quobject
    return np.trace(ρ.reshape(n,m,n,m), axis1=0, axis2=2)
"""

def get_reduced_density_matrix(Psi, NN, trace_out="qpc"):
    """
    Calculate the partial trace from a STATEVECTOR Psi.
    Parameters:
    -----------
    Psi : array
        State vector of the full system
    NN : integer
        Total size of the lattice (L_qpc + 2)
    """
    n_qpc = NN - 2  # QPC sites
    n_qubit = 2     # Qubit sites
    
    # Full density matrix from state vector
    ρ_full = np.outer(Psi, np.conj(Psi.T))
    
    if trace_out == "qpc":
        # Trace out QPC sites, return reduced rho for qubit (2x2)
        return np.trace(ρ_full.reshape(n_qpc, n_qubit, n_qpc, n_qubit), axis1=0, axis2=2)
    
    elif trace_out == "qubit":
        # Trace out qubit sites, return reduced rho for QPC (L_qpc x L_qpc)
        return np.trace(ρ_full.reshape(n_qpc, n_qubit, n_qpc, n_qubit), axis1=1, axis2=3)
    
    else:
        raise ValueError("trace_out must be 'qpc' or 'qubit'")

# Main

In [58]:
# build gaussian wavepacket
alphas = np.exp(-0.5*(x - centered_at)**2 / (Delta**2)) * np.exp(1j * K0 *(x-centered_at))
norm_ = np.linalg.norm(alphas)
alphas = alphas/norm_

if ddot == "momentum":
    print("mom")
    #this is so we get the same qubit state when the qpc hits the bond
    a0, b0 = get_DD_init_for_momentum(K0, af)
    
elif ddot == "free":
    a0 = af
    b0 = bf
elif ddot == "fixed":
    a0, b0 = get_DD_init_for_fixed_orbit(K0,af, phi)

elif ddot == "old":
    a0, b0 = get_DD_init_for_fixed_k(K0)
else:
    print("Invalid Initial state for the qubit")

psi_0 = np.kron(alphas, [a0,b0])

# build the projection opeartor to the qubit-symmetric sector of the hilbert space
Id_qpc = np.eye(L_qpc)
plusket = np.asarray([1/np.sqrt(2),1/np.sqrt(2)])
p_ketbra = np.outer(plusket,plusket)
Psym = np.eye(2*L_qpc) - np.kron(Id_qpc, p_ketbra) # tensor product

# build the momentum values for the bands
k_single_band = np.arange(1,L_qpc+1)*np.pi/(L_qpc+1)

np.shape(psi_0)

mom


(200,)

In [55]:
# Build the hamiltonian and diagonalize it
H_matrix, Hdeco = create_hamiltonians(L_qpc,t, bond_index)

# for the interacting hamiltonian
energies, eigen_vecs = np.linalg.eig(H_matrix)
# normalize
eigen_vecs = eigen_vecs/ np.linalg.norm(eigen_vecs, axis=0)
# Calculate the non-interacting energies and eigenvectors
free_energies, free_eigen_vecs = np.linalg.eig(Hdeco)
# normalize
free_eigen_vecs = free_eigen_vecs/ np.linalg.norm(free_eigen_vecs, axis=0)
sorted_indices, over_matrix = sort_by_overlap_matrix(energies, free_eigen_vecs,eigen_vecs)



In [56]:
# now sort the freee case into bands according to projection
mindices, pindices = sort_by_projection(free_energies,free_eigen_vecs, Psym)

free_energies_m, free_states_m, free_energies_p, free_states_p = get_bands(free_energies, free_eigen_vecs, mindices, pindices)
# now sort the coupled eigenvectors and values according to max overlaps so they match with the free case
sorted_e = energies[sorted_indices]
sorted_vecs = eigen_vecs[:,sorted_indices]
over_energies_m, over_states_m, over_energies_p, over_states_p = get_bands(sorted_e, sorted_vecs, mindices, pindices)


In [ ]:
time_range = np.linspace(0, maxt_time, N_timepoints)

trajectories = np.zeros((L_qpc, len(time_range)))
qubit_traj = []
rho_list = []
St = []

i = 0
for Tau in time_range:
    psi_t = time_evolve(psi_0, Tau,eigen_vecs, energies)
    occupations = get_QPC_occupations(psi_t)
    qubit_traj.append(get_qubit_occupations(psi_t))
    # reduced density matrix 
    rhot = get_reduced_density_matrix(psi_t, L_qpc+2, trace_out="qpc")
    # entropy from schmidt
    #U, S, Vh = schmidt_qubit_qpc(psi_t)
    St.append(entropy_vn(Qobj(rhot), sparse=False) )
    schmis = S**2
    # save in arrays
    trajectories[:,i] = occupations
    rho_list.append(rhot)
    #St.append(-1*np.sum(schmis*np.log(schmis+1e-17))) # avoid log(0)
    St.append(entropy_vn(Qobj(rhot), sparse=False) )
    
    i += 1

In [52]:
param_dict = {"L_qpc": L_qpc, "Omega": Omega, "t":t ,"J":J[0] ,"Jp": J_prime, "bond_index" : bond_index, 
              "K0": K0, "X0":centered_at, "Spread":Delta, "maxt_time": maxt_time, "del_tau":time_range[1]-time_range[0],
             "qubit_init":ddot,  "Re_qubit_0":np.real(a0), "Im_qubit_0":np.imag(a0), 
              "Re_qubit_1":np.real(b0), "Im_qubit_1":np.imag(b0), "phi":phi }

file_name = "exact_L{}_J{}_t{}_om{}_Del{}_xo{}_k{:.4f}_bindex{}_maxtau{:.3f}_tstep{:.3f}_alpha{:.3f}_beta{:.3f}_phi{}.h5".format(
                                    L_qpc, J_prime, t, Omega, Delta,centered_at , K0, bond_index,maxt_time,
                                    time_range[1]-time_range[0], np.abs(a0)**2, np.abs(b0)**2, phi)

results_file = h5py.File(data_route+file_name,'w')
# save parameters and maybe other meta data
grp = results_file.create_group("metadata")
grp.create_dataset("parameters", data=json.dumps(param_dict))

# save the quantities that we are interested in 
grp = results_file.create_group("results")
grp.create_dataset("time", data=time_range )
grp.create_dataset("trajectories", data=trajectories, compression="gzip",compression_opts=9)
grp.create_dataset("d0_density", data=np.asarray(qubit_traj), compression="gzip",compression_opts=9)
grp.create_dataset("qubit_rho", data=rho_list, compression="gzip",compression_opts=9)
grp.create_dataset("Entropy", data=St, compression="gzip",compression_opts=9)

results_file.close()
gc.collect()

0

In [51]:
Param_dict, Times, Occupations, D0_density, Qubit_rho, Entropy = load_exact_diag(data_route, file_name)